In [1]:
import numpy as np
# from pymoo.core.problem import Problem
from pymoo.algorithms.moo.nsga2 import NSGA2
from pymoo.optimize import minimize
from pymoo.core.problem import ElementwiseProblem
# from pymoo.algorithms.soo.nonconvex.pso import PSO
from scipy.interpolate import griddata
import pandas as pd


In [2]:
jpeg_cha = pd.read_csv("../../stable_tests/measurements/jpeg_snr_cha/characteristic.csv")

jpeg_cha = jpeg_cha[jpeg_cha["sparsity"]>0]
jpeg_cha = jpeg_cha[jpeg_cha["datasize_est"]>0]
tensor_size = 128*26*26 *4
jpeg_cha["ratio"] = tensor_size/jpeg_cha["datasize_est"]

In [3]:
pruning = [0.05, 0.1, 0.15, 0.2, 0.25,0.3,0.35]
quality = [60,70,80,90,100]

sample_points=[]
cmp_sample_values = []
snr_sample_values = []

cha_df_group =jpeg_cha.groupby("pruning_thresh")
for p in pruning:
    cha_df = cha_df_group.get_group(p)
    cha_quality_df = cha_df.groupby("quality")
    for q in quality:
        cha_plot_df= cha_quality_df.get_group(q)
        sample_points.append([q,p])
        cmp_sample_values.append(cha_plot_df["ratio"].mean())
        snr_sample_values.append(cha_plot_df["reconstruct_snr"].mean())

sample_points = np.array(sample_points)
cmp_sample_values = np.array(cmp_sample_values)
snr_sample_values = np.array(snr_sample_values)

In [5]:
# Interpolate the constraint function
def interpolated_constraint_cmp(xy):
    return griddata(sample_points, cmp_sample_values, (xy[0], xy[1]/100), method='cubic')

def interpolated_constraint_snr(xy):
    return griddata(sample_points, snr_sample_values, (xy[0], xy[1]/100), method='cubic')


class CustomProblem(ElementwiseProblem):

    def __init__(self,snr,cmp):
        self.snr = snr
        self.cmp = cmp
        super().__init__(n_var=2, n_obj=2, n_ieq_constr=2, xl=[60,5], xu=[100,35])

    def _evaluate(self, x, out, *args, **kwargs):
        out["F"] = [x[0],-x[1]]
        out["G"] = np.array([
            self.cmp-interpolated_constraint_cmp(x),
            self.snr-interpolated_constraint_snr(x), 
        ])
        # penalty = 1000  # Large penalty factor
        # if constraint_1 > 0:  # If constraint is violated
        #     out["F"] += penalty * constraint_1

In [9]:
# Initialize the problem
problem = CustomProblem(snr=5,cmp=30)

# Select the algorithm (Genetic Algorithm in this case)
algorithm = NSGA2(pop_size=10)
# Perform the optimization
result = minimize(problem, algorithm, termination=('n_gen', 10),verbose=True)

# # Get and display the results
# optimal_x, optimal_y = result.X
# optimal_objective_value = -result.F[0]

print("Best solution found: \nX = %s\nF = %s" % (result.X, result.F))

n_gen  |  n_eval  | n_nds  |     cv_min    |     cv_avg    |      eps      |   indicator  
     1 |       10 |      2 |  0.000000E+00 |  4.2593215393 |             - |             -
     2 |       20 |      4 |  0.000000E+00 |  0.4251988124 |  0.0667683013 |         ideal
     3 |       30 |      5 |  0.000000E+00 |  0.000000E+00 |  0.0534425977 |             f
     4 |       40 |      4 |  0.000000E+00 |  0.000000E+00 |  0.0059761193 |         ideal
     5 |       50 |      4 |  0.000000E+00 |  0.000000E+00 |  0.0338828464 |             f
     6 |       60 |      3 |  0.000000E+00 |  0.000000E+00 |  0.0064401071 |         ideal
     7 |       70 |      6 |  0.000000E+00 |  0.000000E+00 |  0.0035544399 |         ideal
     8 |       80 |      6 |  0.000000E+00 |  0.000000E+00 |  0.0671430491 |         ideal
     9 |       90 |      7 |  0.000000E+00 |  0.000000E+00 |  0.0512389670 |         nadir
    10 |      100 |      6 |  0.000000E+00 |  0.000000E+00 |  0.0633628459 |         nadir

In [10]:
interpolated_constraint_cmp(result.X)

array([nan, nan])

In [ ]:
interpolated_constraint_snr(result.X)

array(35.16830948)